In [1]:
import glob
import numpy as np
import rasterio
from rasterio.merge import merge
import geopandas as gpd
import matplotlib.pyplot as plt


In [ ]:
def merge_tiles(tile_dir, out_tif="merged.tif"):
    tile_paths = sorted(glob.glob(f"{tile_dir}/*.tif"))
    assert len(tile_paths) > 0, "No tiles found!"

    srcs = [rasterio.open(p) for p in tile_paths]

    mosaic, transform = merge(srcs)

    meta = srcs[0].meta.copy()
    meta.update({
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": transform
    })

    with rasterio.open(out_tif, "w", **meta) as dst:
        dst.write(mosaic)

    for s in srcs:
        s.close()

    print(f"Merged raster saved: {out_tif}")
    return out_tif


In [ ]:
# def overlay_polygons_png(
#     raster_path,
#     polygon_path,
#     out_png="overlay.png",
#     max_size=2048,
#     polygon_color="red",
#     polygon_linewidth=1.5
# ):
#     # --- Load raster ---
#     with rasterio.open(raster_path) as src:
#         img = src.read([1, 2, 3])  # RGB
#         transform = src.transform
#         crs = src.crs

#     img = np.transpose(img, (1, 2, 0))
#     img = (img - img.min()) / (img.max() - img.min())

#     # --- Load polygons ---
#     gdf = gpd.read_file(polygon_path)

#     if gdf.crs != crs:
#         gdf = gdf.to_crs(crs)

#     # --- Downscale for visualization ---
#     h, w = img.shape[:2]
#     scale = min(max_size / w, max_size / h, 1.0)

#     if scale < 1.0:
#         img = img[::int(1/scale), ::int(1/scale)]

#     # --- Plot ---
#     fig, ax = plt.subplots(figsize=(12, 12))
#     ax.imshow(img)

#     gdf.plot(
#         ax=ax,
#         facecolor="none",
#         edgecolor=polygon_color,
#         linewidth=polygon_linewidth
#     )

#     ax.set_axis_off()
#     plt.tight_layout()
#     plt.savefig(out_png, dpi=300, bbox_inches="tight")
#     plt.close()

#     print(f"Overlay PNG saved: {out_png}")


In [ ]:
# tile_dir = "tiles_2048"            # folder with your 2048x2048 tiles
# polygon_geojson = "polygon.geojson"

# merged_tif = merge_tiles(tile_dir, out_tif="merged.tif")

# overlay_polygons_png(
#     raster_path=merged_tif,
#     polygon_path=polygon_geojson,
#     out_png="merged_with_polygons.png"
# )


In [5]:
import rasterio
from rasterio.enums import Resampling
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt

# -------------------------------
# 1. Open raster
# -------------------------------
file_path = "/home/basnyat/shared/Hanumannagar_Preflood_Orthomosaic.tif"
scale = 0.05  # 5% of original size to keep visible values

with rasterio.open(file_path) as src:
    # Read 3 bands (RGB)
    bands_to_read = [1,2,3]
    out_shape = (
        len(bands_to_read),
        int(src.height*scale),
        int(src.width*scale)
    )
    img = src.read(bands_to_read, out_shape=out_shape, resampling=Resampling.average)
    # Adjust transform for downsampled image
    transform = src.transform * src.transform.scale(
        (src.width/out_shape[2]),
        (src.height/out_shape[1])
    )
    raster_crs = src.crs
    raster_bounds = box(*src.bounds)

# -------------------------------
# 2. Load vector data
# -------------------------------
gdf = gpd.read_file("all_roads_combined.geojson")
if gdf.crs != raster_crs:
    gdf = gdf.to_crs(raster_crs)

# Clip to raster bounds
gdf_clip = gpd.clip(gdf, raster_bounds)

# -------------------------------
# 3. Plot using rasterio.show and overlay polygons
# -------------------------------
fig, ax = plt.subplots(figsize=(12,12))
show(img, transform=transform, ax=ax)  # rasterio handles normalization for RGB

# Overlay vector polygons
gdf_clip.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1)

ax.axis('off')
plt.tight_layout()

# -------------------------------
# 4. Save PNG
# -------------------------------
output_png = "orthomosaic_with_roads.png"
plt.savefig(output_png, dpi=300, bbox_inches='tight', pad_inches=0)
plt.close(fig)

print(f"Saved overlay PNG at: {output_png}")


Saved overlay PNG at: orthomosaic_with_roads.png
